In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch.nn as nn
import torch
import numpy as np

class GAN_Rec(torch.nn.Module):

    def __init__(self, number_of_items):
      super().__init__()

      self.generator = nn.Sequential(
            nn.Linear(number_of_items, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, number_of_items),
            nn.Sigmoid()
          )


      self.discriminator = nn.Sequential(
            nn.Linear(number_of_items, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def generate(self, user_data, mask = True):
      generated_recs = self.generator(user_data)

      if mask == False:
        return generated_recs * 5

      masked_recs = torch.mul(generated_recs, user_data)
      return masked_recs

    def discriminate(self, masked_recs):
      return self.discriminator(masked_recs)

    def d_loss(self, u_data):
      u_data = u_data[0]
      dis_real = self.discriminate(u_data)
      dis_fake = self.discriminate(self.generate(u_data))
      return -torch.mean(torch.log(1. - dis_fake) + torch.log(dis_real))

    def g_loss(self, user_Data):
      user_Data = user_Data[0]
      masked_recs = self.generate(user_Data)
      dis_output = self.discriminate(masked_recs)
      return -torch.mean(torch.log(dis_output))

In [ ]:
def train_GANs_REC(model, dataloader, epochs, learning_rate, device = 'cuda'):

  model.to(device)
  model.train()

  optimizer_g = torch.optim.Adam(model.generator.parameters(), lr=learning_rate)
  optimizer_d = torch.optim.Adam(model.discriminator.parameters(), lr=learning_rate)
  for epoch in range(epochs):
    for user_data in dataloader:

      user_data = user_data.to(device)

      optimizer_d.zero_grad()
      d_loss = model.d_loss([user_data])
      d_loss.backward()
      optimizer_d.step()


      optimizer_g.zero_grad()
      g_loss = model.g_loss([user_data])
      g_loss.backward()
      optimizer_g.step()

    print(f"Epoch [{epoch+1}/{epochs}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

In [ ]:
def test_GAN(model, dataloader, device = 'cuda'):
  model.to(device)
  predicted_recs = []
  model.eval()
  with torch.no_grad():
    for user_data in dataloader:
      user_data = user_data.to(device)
      recs = model.generate(user_data, False)
      recs = recs.squeeze().tolist()
      predicted_recs.append(recs)
  return predicted_recs

In [ ]:
import pandas as pd
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

data = pd.read_csv( '/content/drive/MyDrive/LLM Recommender Research project/Coding Resources/Baseline Models/MovieLen1M/ratings.dat',
                   sep = '::', header = None, engine = 'python',
                   names = ['UserIDs', 'MovieIDs', 'Ratings'],
                    usecols = [0,1,2])


data = data.pivot_table(index = "UserIDs", columns = "MovieIDs", values = "Ratings")

data = data.fillna(0)
data = data.to_numpy()

train_data, test_data = train_test_split(data, test_size = 0.2)

train_data = torch.tensor(train_data, dtype=torch.float32)
test_data = torch.tensor(test_data, dtype=torch.float32)

train_loader = DataLoader(train_data, batch_size = 16, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 1, shuffle = True)

In [ ]:
model = GAN_Rec(len(data[0]))
train_GANs_REC(model, train_loader, 100, 1e-4)
predicted_recs = test_GAN(model, test_loader)

Epoch [1/100], D Loss: 1.3854, G Loss: 0.7234
Epoch [2/100], D Loss: 1.3896, G Loss: 0.7638
Epoch [3/100], D Loss: 1.4121, G Loss: 0.6727
Epoch [4/100], D Loss: 1.4157, G Loss: 0.6649
Epoch [5/100], D Loss: 1.4308, G Loss: 0.6980
Epoch [6/100], D Loss: 1.3718, G Loss: 0.6867
Epoch [7/100], D Loss: 1.3825, G Loss: 0.7053
Epoch [8/100], D Loss: 1.3754, G Loss: 0.7169
Epoch [9/100], D Loss: 1.4172, G Loss: 0.7023
Epoch [10/100], D Loss: 1.4284, G Loss: 0.7377
Epoch [11/100], D Loss: 1.4154, G Loss: 0.7240
Epoch [12/100], D Loss: 1.3668, G Loss: 0.6722
Epoch [13/100], D Loss: 1.4154, G Loss: 0.6785
Epoch [14/100], D Loss: 1.3568, G Loss: 0.6970
Epoch [15/100], D Loss: 1.3603, G Loss: 0.7058
Epoch [16/100], D Loss: 1.3939, G Loss: 0.6913
Epoch [17/100], D Loss: 1.3872, G Loss: 0.7100
Epoch [18/100], D Loss: 1.3630, G Loss: 0.7155
Epoch [19/100], D Loss: 1.3637, G Loss: 0.7140
Epoch [20/100], D Loss: 1.4036, G Loss: 0.6812
Epoch [21/100], D Loss: 1.3854, G Loss: 0.6907
Epoch [22/100], D Loss

In [ ]:
from collections import defaultdict

def get_predictions(rating_matrix, recon_rating_matrix):
  predictions = []
  for i in range(len(rating_matrix)):
    for j in range(len(rating_matrix[0])):
      t = (i, rating_matrix[i][j], recon_rating_matrix[i][j])
      predictions.append(t)
  return predictions

def precision_recall_at_k(predictions, k=100, threshold=3.0):

    user_est_true = defaultdict(list)
    for uid, true_r, est in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

        precision = sum(rating for rating in precisions.values()) / len(precisions)
        recall = sum(rating for rating in recalls.values()) / len(recalls)
    return precision, recall

def f1_score(precision, recall):
    return 2 * (precision * recall) / (precision + recall)

In [ ]:
predictions = get_predictions(test_data.numpy(), predicted_recs)
precision, recall = precision_recall_at_k(predictions)
f1 = f1_score(precision, recall)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precision: 0.035033112582781495
Recall: 0.02583352879313141
F1 Score: 0.029738093055959293
